Cykl Eulera
</br>
$def.$ Cykl Eulera w grafie $G$, to taki cykl, który przez każdą krawędź $G$ przechodzi dokładnie raz.
</br>
$tw.$ Graf nieskierowany i spójny posiada cykl Eulera $\iff$ każdy wierzchołek ma parzysty stopień.

<ul>Propozycja algorytmu:
    <li>wykonujemy DFS, ale po drodzie 'usuwamy' krawędzie, po których przeszliśmy</li>
    <li>do danego wierzchołka możemy wejść dowolną liczbę razy</li>
    <li>po przetworzeniu wierzchołka dopisujemy go na początku tworzonego cyklu</li>
</ul>

Złożoność $O(V+E)$ lub $O(V^2)$
</br>-> łatwo wpaść w $O(V^2 + VE)$ lub $O(V^3)$, aby tego uniknąć należy mieć "wskaźnik" do ostatniego wybranego sąsiada, w innym wypadku, każdy wybór sąsiada ma złożoność $O(n)$

In [1]:
def EulerCircuit(G,s):
    n=len(G)
    lastEdge=[[False for _ in range(n)] for _ in range(n)]
    Euler=[]
    def DFSVisit(G,v):
        nonlocal lastEdge, Euler
        for u in G[v]:
            if not lastEdge[v][u]:
                lastEdge[v][u],lastEdge[u][v]=True,True
                DFSVisit(G,u)
        Euler.append(v)
    DFSVisit(G,s)
    return Euler

G=[[1,2],[0,2],[0,1,3,4],[2,4],[2,3]]

print(EulerCircuit(G,0))
# Działa, ale z macierzą pomocniczą? sus

[0, 2, 4, 3, 2, 1, 0]


Cykl Hamiltona
</br>
$def.$ Cykl Hamiltona, to cykl prosty (niepowtarzający wierzchołków), który odwiedza każdy wierzchołek dokładnie raz.

Propozycja Algorytmu: Brute-force - sprawdza $O(V!)$ możliwości.
</br>
Nie da się wiele lepiej, bo rozpoznawanie czy graf ma cykl Hamiltona jest problemem NP-zupełnym

Silnie Spójne składowe w grafie skierowanym
</br>
$def.$ Niech $G=(V,E)$ będzie pewnym grafiem skierowanym. </br>
Mówimy, że $u,v \in V $ należą do tej samej silnie spójniej składowej jeśli istnieje ścieżka z $u$ do $v$ i $v$ do $u$
</br>

Graf składający się z silnych spójnych składowych jest DAGiem, inaczej, byłby po prostu jedną spójną składową 

Propozycja algorytmu:
 - wykonujemy DFS na $G$, zapisując czasy przetworzenia   
 - odwracamy kierunek wszystkich krawędzi 
 - wykonujemy DFS, wybierając startowe wierzchołki w kolejności malejących względem czasu przetworzenia   

Znajdowanie mostów w grafach nieskierowanych

$def.$ Most, to krawędź $e$ w grafie nieskierowanym $G$, jeśli usunięcie jej rozspójnia graf</br>
$tw.$ Krawędź $e$ jest mostem $\iff$ nie leży na żadnym cyklu prostym w grafie</br>
$dow.:$</br>$e$ jest mostem $\Rightarrow$ nie leży na cyklu</br> $e$ nie leży na żadnym cyklu $\Rightarrow$ jest mostem</br>

Algorytm:
 - wykonujemy DFS, dla  każdego $v$ zapisz jego czas odwiedzenia $d(v)$   
 - dla każdego $v$ oblicz $low(v) = min(d(v), d(u), min(low(w)))$ min z $v$ jest krawędź wsteczna do $u$, $w$ to dziecko $v$ w drzewie DFS
 - Mosty to krawędzie ${v,p(v)}$, takie gdzie czas d(v) = low(v)  S

